In [5]:
#open pickle file with dataframe
import pickle
import pandas as pd
with open('../datafiles/activeDamsPercentileRankDf.pkl', 'rb') as f:
    df = pickle.load(f)


In [6]:
df.head()

,Categories,CED,BW,WW,YW,MK,TM,Growth
0,Num Animals,8845,8845,8845,8845,8845,8845,8845
1,High,15,9.3,75,122,35,72,131.80
2,Average,-1,1.7,43,79,26,47,79.50
3,Low,-14,-5.9,18,45,12,25,34.03
4,1%,9,-1.6,56,96,31,57,106.44


In [13]:
df.loc[df['Categories'] == "High"]["CED"][1]

'15'